# NDEx2 Tutorial

This tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires the Python 3.6+ and ndex2 module, see the NDEx Client Tutorial for installation instructions.

## Importing Packages

In [1]:
from nicecxModel.NiceCXNetwork import NiceCXNetwork
from nicecxModel.cx.aspects.NodesElement import NodesElement
from nicecxModel.cx.aspects.EdgesElement import EdgesElement
from nicecxModel.cx.aspects.NodeAttributesElement import NodeAttributesElement
from nicecxModel.cx.aspects.EdgeAttributesElement import EdgeAttributesElement
import networkx as nx
import pandas as pd

## Five Ways to Create a NiceCX network

1. An empty network. 
          niceCx = NiceCXNetwork()
2. Using a cx file. 
          niceCx = NiceCXNetwork(cx=cx)
3. Loading it from an NDEx server.
          niceCx = NiceCXNetwork(server='test.ndexbio.org', uuid='01c83ba5-0d90-11e6-b550-06603eb7f303')
4. Using NetworkX.
          niceCx = NiceCXNetwork(networkx_G=G)
5. Using a Pandas DataFrame.
          niceCx = NiceCXNetwork(pandas_df=df)



## Create an empty NiceCx network


In [2]:
#Create an empty niceCx network
niceCx_creatures = NiceCXNetwork()

### Populate the Network

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute.Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [3]:
niceCx_creatures.setName("Food Web")

### Add Nodes and Edges

In [4]:
fox_node = niceCx_creatures.addNode(node_name='Fox')
mouse_node = niceCx_creatures.addNode(node_name='Mouse')
bird_node = niceCx_creatures.addNode(node_name='Bird')

fox_bird_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = niceCx_creatures.addEdge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')


The addNode and addEdge methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

### Add Attributes

The addNodeAttribute and addEdgeAttribute require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [5]:
niceCx_creatures.addNodeAttribute(property_of=fox_node, name='Color', values='Red')

niceCx_creatures.addNodeAttribute(property_of=mouse_node, name='Color', values='Gray')

niceCx_creatures.addNodeAttribute(property_of=bird_node, name='Color', values='Blue')

In [6]:
niceCx_creatures.addEdgeAttribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [7]:
print(niceCx_creatures.getSummary())

Name: Food Web
Nodes: 3
Edges: 2
Node Attributes: 3
Edge Attributes: 1



# Create a NiceCX network using a CX file

In [8]:
niceCx = NiceCXNetwork(filename='SimpleNetwork.cx')
print(niceCx.getSummary())

Name: 
Nodes: 2
Edges: 1
Node Attributes: 0
Edge Attributes: 0



# Create a NiceCX network from a NetworkX network

In [2]:
G_small = nx.Graph()
G_small.add_node('ABC')
G_small.add_node('DEF')
G_small.add_edges_from([('ABC','DEF')])

niceCx_networkx = NiceCXNetwork(networkx_G=G_small)

Networks can be manipulated using NetworkX and later turned into niceCx.

In [2]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

Use networkx to create a subgraph of the shortest path between nodes ABC and MNO

In [3]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

H = G.subgraph(short_path)

Build a niceCx object from the full networkx graph and the sub-graph.

In [5]:
#====================================
# BUILD NICECX FROM NETWORKX GRAPHS
#====================================
niceCx_full = NiceCXNetwork(networkx_G=G)
niceCx_full.setName('Created from networkx (full)')

niceCx_short = NiceCXNetwork(networkx_G=H)
niceCx_short.setName('Created from networkx (shortest path)')
                       
#=============================
# PRINT THE NETWORK SUMMARIES
#=============================
print(niceCx_full.getSummary())
print(G.edges())
print('')

print(niceCx_short.getSummary())
print(H.edges())

Name: Created from networkx (full)
Nodes: 7
Edges: 8
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('XYZ', 'MNO'), ('JKL', 'GHI'), ('JKL', 'DEF'), ('JKL', 'MNO'), ('PQR', 'DEF'), ('MNO', 'DEF'), ('GHI', 'DEF')]

Name: Created from networkx (shortest path)
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0

[('ABC', 'DEF'), ('DEF', 'MNO')]


# Create using Pandas DataFrame

Create a NiceCx model using Pandas DataFrame

### Using a 2 column dataframe with no headers:

In [9]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

#==============================================
# BUILD NICECX FROM PANDAS DATAFRAME 2-Column
#==============================================
niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx.getSummary())

Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0



### Using a 3 column dataframe with no headers:

In [10]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

#==============================================
# BUILD NICECX FROM PANDAS DATAFRAME 3-Column
#==============================================
niceCx = NiceCXNetwork(pandas_df=df)

print(niceCx.getSummary())

Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 0



### Using 3+ columns plus headers to specify attributes columns

In [11]:
df = pd.DataFrame.from_items([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

niceCx = NiceCXNetwork()
#==================================================
# BUILD NICECX FROM PANDAS DATAFRAME WITH HEADERS
#==================================================
niceCx.create_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

print(niceCx.getSummary())

0
Nodes: 3
Edges: 2
Node Attributes: 0
Edge Attributes: 2



# Create using network hosted on NDEx server

In [12]:
niceCx = NiceCXNetwork(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
print(niceCx.getSummary())


http://public.ndexbio.org/v2/network/f1dd6cc3-0007-11e6-b550-06603eb7f303/aspect
Nodes: 36
Edges: 37
Node Attributes: 778
Edge Attributes: 659

